In [1]:
import os

In [2]:
%pwd

'e:\\DATA SCIENCE\\INSAID\\CNNPROJECTS\\kidney-disease-classification-deep-learning-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\DATA SCIENCE\\INSAID\\CNNPROJECTS\\kidney-disease-classification-deep-learning-project'

In [5]:
# export MLFLOW_TRACKING_URI=https://dagshub.com/prashantsundge/kidney-disease-classification-deep-learning-project.mlflow
# export MLFLOW_TRACKING_USERNAME=prashantsundge 
# export MLFLOW_TRACKING_PASSWORD=3ba21498f90817599ad55abbe94186bb521b3d22


In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/prashantsundge/kidney-disease-classification-deep-learning-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="prashantsundge"
os.environ["MLFLOW_TRACKING_PASSWORD"]="3ba21498f90817599ad55abbe94186bb521b3d22"


In [7]:
import tensorflow as tf


In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict
    mlflow_url : str
    params_image_size : list
    params_batch_size : int

In [10]:
from cnnClassifier.constants import * 
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
          eval_config = EvaluationConfig(
                path_of_model = "artifacts/training/model.h5",
                training_data = "artifacts/data_ingestion/kidney-ct-scan-image",
                mlflow_url = "https://dagshub.com/prashantsundge/kidney-disease-classification-deep-learning-project.mlflow",
                all_params = self.params, 
                params_image_size = self.params.IMAGE_SIZE,
                params_batch_size = self.params.BATCH_SIZE
          )

          return eval_config


In [12]:
import tensorflow as tf
from pathlib import Path 
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [13]:
class Evaluation:
    def __init__(self, config : EvaluationConfig):
        self.config = config


    
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"

        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs

        )
    @staticmethod
    def load_model(path :Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def save_score(self):
        scores = {"loss" : self.score[0], "accuracy" : self.score[1]}
        save_json(path = Path("scores.json"), data= scores)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        

    def save_score(self):
        scores = {"loss" : self.score[0], "accuracy" : self.score[1]}
        save_json(path = Path("scores.json"), data= scores)

    #mlflow code
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss" : self.score[0], "accuracy" : self.score[1]}
            )
            #model registry does not work with file store
            if tracking_url_type_store != "file" :
                #register the model
                #there are other ways to use the model Registry, Which depends on the use case,
                #Please refer to the doc for more information 
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name= "VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [15]:
#initialise pipeline

try :
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-04-05 13:07:53,082:INFO: common:yaml file: config\config.yaml loaded sucessfully]
[2024-04-05 13:07:53,167:INFO: common:yaml file: params.yaml loaded sucessfully]
[2024-04-05 13:07:53,170:INFO: common:Created directory at : artifacts]
Found 150 images belonging to 2 classes.
10/10 [==============================] - 53s 5s/step - loss: 0.8751 - accuracy: 0.5733


2024/04/05 13:08:49 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-04-05 13:08:53,325:WARNING: save:Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmpcw37_0we\model\data\model\assets
[2024-04-05 13:08:54,998:INFO: builder_impl:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmpcw37_0we\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/04/05 13:10:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 5
Created version '5' of model 'VGG16Model'.
